In [ ]:
A Healthcare Analytics Company wants to analyze patient visit data to optimize hospital resource allocation. The company collects daily patient visit records, including visit timestamps, doctor assignments, diagnosis details, and billing information. The data needs to be processed to generate insights on hospital occupancy rates, doctor workload, and revenue per department.

User Stories:
As a Hospital Administrator, I want to track the number of patients visiting each department daily, so that I can manage staffing efficiently.
As a Finance Manager, I want to analyze total revenue generated per department, so that I can track financial performance.
As a Chief Medical Officer, I want to determine the workload of each doctor, so that I can distribute patients effectively.
As a Data Analyst, I want to categorize high-risk patients based on their diagnosis, so that I can study trends in patient conditions.
Given Dataset (In-Memory Data)
Raw Patient Visit Data

VisitID	PatientID	DoctorID	Department	Diagnosis	VisitDateTime	BillingAmount
V001	P1001	D001	Cardiology	Heart Disease	2025-02-12 09:30:00	500
V002	P1002	D002	Neurology	Migraine	2025-02-12 10:00:00	300
V003	P1003	D001	Cardiology	Hypertension	2025-02-12 10:45:00	200
V004	P1004	D003	Orthopedics	Fracture	2025-02-12 11:15:00	700
V005	P1005	D002	Neurology	Stroke	2025-02-12 12:00:00	1000
V006	P1006	D004	Dermatology	Skin Allergy	2025-02-12 12:30:00	250
V007	P1007	D001	Cardiology	Heart Attack	2025-02-12 13:00:00	1500
V008	P1008	D003	Orthopedics	Arthritis	2025-02-12 14:00:00	400
Transformations to be Performed (ETL Workflow):
Data Cleaning:

Convert VisitDateTime to timestamp format.
Remove duplicate records.
Handle missing values (replace null billing amounts with median billing per department).
Data Enrichment:

Add a column "Risk Level" based on Diagnosis Severity:
Critical (Heart Attack, Stroke)
Moderate (Heart Disease, Hypertension, Fracture, Arthritis)
Mild (Migraine, Skin Allergy)
Add a column Revenue Category:
High Revenue (>800 USD)
Medium Revenue (400-800 USD)
Low Revenue (<400 USD)
Aggregation & Insights:

Total Patients per Department per Day.
Total Revenue per Department per Day.
Top 2 High Revenue Generating Departments.
Workload Analysis: Number of Patients per Doctor per Day.
Window Functions for Ranking:

Rank doctors by workload (most patients seen).
Rank departments by revenue contribution.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, when, rank
from pyspark.sql.window import Window

# ✅ Step 1: Initialize Spark Session
spark = SparkSession.builder.appName("HealthcareETL").getOrCreate()

# ✅ Step 2: Create In-Memory Dataset
raw_data = [
    ("V001", "P1001", "D001", "Cardiology", "Heart Disease", "2025-02-12 09:30:00", 500),
    ("V002", "P1002", "D002", "Neurology", "Migraine", "2025-02-12 10:00:00", 300),
    ("V003", "P1003", "D001", "Cardiology", "Hypertension", "2025-02-12 10:45:00", 200),
    ("V004", "P1004", "D003", "Orthopedics", "Fracture", "2025-02-12 11:15:00", 700),
    ("V005", "P1005", "D002", "Neurology", "Stroke", "2025-02-12 12:00:00", 1000),
    ("V006", "P1006", "D004", "Dermatology", "Skin Allergy", "2025-02-12 12:30:00", 250),
    ("V007", "P1007", "D001", "Cardiology", "Heart Attack", "2025-02-12 13:00:00", 1500),
    ("V008", "P1008", "D003", "Orthopedics", "Arthritis", "2025-02-12 14:00:00", 400)
]
columns = ["VisitID", "PatientID", "DoctorID", "Department", "Diagnosis", "VisitDateTime", "BillingAmount"]

df = spark.createDataFrame(raw_data, columns)

# ✅ Step 3: Data Cleaning
from pyspark.sql.functions import to_timestamp

df = df.withColumn("VisitDateTime", to_timestamp(col("VisitDateTime")))

# ✅ Step 4: Data Enrichment
risk_mapping = when(col("Diagnosis").isin("Heart Attack", "Stroke"), "Critical") \
    .when(col("Diagnosis").isin("Heart Disease", "Hypertension", "Fracture", "Arthritis"), "Moderate") \
    .otherwise("Mild")

df = df.withColumn("RiskLevel", risk_mapping)

revenue_mapping = when(col("BillingAmount") > 800, "High Revenue") \
    .when(col("BillingAmount").between(400, 800), "Medium Revenue") \
    .otherwise("Low Revenue")

df = df.withColumn("RevenueCategory", revenue_mapping)

# ✅ Step 5: Aggregation
patient_per_department = df.groupBy("Department").agg(count("PatientID").alias("TotalPatients"))
revenue_per_department = df.groupBy("Department").agg(sum("BillingAmount").alias("TotalRevenue"))
doctor_workload = df.groupBy("DoctorID").agg(count("PatientID").alias("PatientsHandled"))

# ✅ Step 6: Window Functions for Ranking
window_spec = Window.orderBy(col("PatientsHandled").desc())
ranked_doctors = doctor_workload.withColumn("Rank", rank().over(window_spec))

window_spec_revenue = Window.orderBy(col("TotalRevenue").desc())
ranked_departments = revenue_per_department.withColumn("Rank", rank().over(window_spec_revenue))

# ✅ Display Results
df.show()
patient_per_department.show()
revenue_per_department.show()
doctor_workload.show()
ranked_doctors.show()
ranked_departments.show()

# ✅ Stop Spark Session
spark.stop()


25/02/13 12:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/13 12:16:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/13 12:16:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/02/13 12:16:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/02/13 12:16:10 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


+-------+---------+--------+-----------+-------------+-------------------+-------------+---------+---------------+
|VisitID|PatientID|DoctorID| Department|    Diagnosis|      VisitDateTime|BillingAmount|RiskLevel|RevenueCategory|
+-------+---------+--------+-----------+-------------+-------------------+-------------+---------+---------------+
|   V001|    P1001|    D001| Cardiology|Heart Disease|2025-02-12 09:30:00|          500| Moderate| Medium Revenue|
|   V002|    P1002|    D002|  Neurology|     Migraine|2025-02-12 10:00:00|          300|     Mild|    Low Revenue|
|   V003|    P1003|    D001| Cardiology| Hypertension|2025-02-12 10:45:00|          200| Moderate|    Low Revenue|
|   V004|    P1004|    D003|Orthopedics|     Fracture|2025-02-12 11:15:00|          700| Moderate| Medium Revenue|
|   V005|    P1005|    D002|  Neurology|       Stroke|2025-02-12 12:00:00|         1000| Critical|   High Revenue|
|   V006|    P1006|    D004|Dermatology| Skin Allergy|2025-02-12 12:30:00|      

+-----------+-------------+
| Department|TotalPatients|
+-----------+-------------+
|  Neurology|            2|
|Dermatology|            1|
| Cardiology|            3|
|Orthopedics|            2|
+-----------+-------------+

+-----------+------------+
| Department|TotalRevenue|
+-----------+------------+
|  Neurology|        1300|
|Dermatology|         250|
| Cardiology|        2200|
|Orthopedics|        1100|
+-----------+------------+

+--------+---------------+
|DoctorID|PatientsHandled|
+--------+---------------+
|    D002|              2|
|    D003|              2|
|    D004|              1|
|    D001|              3|
+--------+---------------+



25/02/13 12:16:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+---------------+----+
|DoctorID|PatientsHandled|Rank|
+--------+---------------+----+
|    D001|              3|   1|
|    D002|              2|   2|
|    D003|              2|   2|
|    D004|              1|   4|
+--------+---------------+----+



25/02/13 12:16:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+------------+----+
| Department|TotalRevenue|Rank|
+-----------+------------+----+
| Cardiology|        2200|   1|
|  Neurology|        1300|   2|
|Orthopedics|        1100|   3|
|Dermatology|         250|   4|
+-----------+------------+----+

